# SocraticFlanT5 - Baseline

## Introduction

The goal of this jupyter notebook is to reproduce the Socratic Models paper with the Flan-T5 model. This provides a baseline for us to build upon.

## Imports

In [1]:
# Package loading
from transformers import set_seed
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Local imports
import sys

sys.path.append('..')
try:
    os.chdir('scripts')
except:
    pass

from scripts.coco_captioning_baseline import ImageCaptionerBaseline

## Instantiate the improved captioner class

In [2]:
image_captioner = ImageCaptionerBaseline(set_type='demo')

load_places starting!
load_places took 0.0s!
load_objects starting!
load_objects took 0.0s!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Run main

In [ ]:
image_captioner.main()

main starting!
generate_response starting!


## Load image and compute image embedding

In [ ]:
image_folder = '../data/images/example_images/'
# Show the image
for img_name in image_captioner.image_manager.demo_names:
    print(f'{img_name}:')
    plt.imshow(image_captioner.image_manager.load_image(image_folder + img_name))
    plt.show()
    cond = image_captioner.generated_caption_df['image_name'] == img_name
    generate_caption = image_captioner.generated_caption_df[cond]['generated_caption'].iloc[0]
    print(f'generate caption: "{generate_caption}"\n')
    print('====================================\n')

In [ ]:
image_captioner.sorted_obj_dic['../data/images/example_images/wedding_image.jpg']

In [ ]:
image_captioner.generated_caption_df[cond]['generated_caption'].iloc[0]